In [1]:
from datasets import load_dataset, DatasetDict, Dataset
import re
from tqdm import tqdm
from icu import Transliterator
from myanmartools import ZawgyiDetector

In [2]:
def zawgyi_to_unicode_converter(examples):
    detector = ZawgyiDetector()
    converter = Transliterator.createInstance('Zawgyi-my')

    texts = examples['text']
    converted_texts = []

    for text in texts:
        score = detector.get_zawgyi_probability(text)
        if score > 0.5:
            text = converter.transliterate(text)

        converted_texts.append(text)

    examples['text'] = converted_texts
    return examples

detector = ZawgyiDetector()
converter = Transliterator.createInstance('Zawgyi-my')


def is_zawgyi(text):
    score = detector.get_zawgyi_probability(text)
    if score > 0.3:
        return 1
    
    return -1


In [3]:
def clean_text(text):
    # Regex that keeps:
    # - ASCII: \u0000–\u007F
    # - Myanmar: \u1000–\u109F
    # - Extended Myanmar (optional): \uAA60–\uAA7F, \uA9E0–\uA9FF, \uAAE0–\uAAFF

    pattern = r"[^\u0000-\u007F\u1000-\u109F\uAA60-\uAA7F\uA9E0-\uA9FF\uAAE0-\uAAFF]"
    cleaned = re.sub(pattern, "", text)
    return cleaned

In [4]:
def cleanup_corpus(example, column):
    raw_text = example[column]
    raw_text = re.sub(r' +', ' ', raw_text)
    raw_text = re.sub(r'\n+\s+', '\n', raw_text)
    return {"clean_text": clean_text(raw_text)}

# 1. Wikipedia Dataset

In [5]:
wiki_ds = load_dataset("chuuhtetnaing/myanmar-wikipedia-dataset")

In [6]:
wiki_ds = wiki_ds.map(cleanup_corpus, fn_kwargs={"column": "content"})

In [7]:
wiki_ds['train'][402]['content']

'EGS-zs8-1 သည် တောင်ဘက် နက္ခတ်တာရာ ဘိုးတီ ကြယ်စုတန်းမှ ဝေးကွာသည့် ဂလက်ဆီဖြစ်သည်။ ယင်းသည် ၂၀၁၅ မေလတွင် အဝေးဆုံးမှ ဂယ်လက်ဆီအဖြစ် ရပ်တည်လျက်ရှိခဲ့သည်။၂၀၁၅ ဂျူလိုင်၂၀ တွင် EGSY-2008532660 ဟူသော ဂလက်ဆီက အဝေးဆုံးအဖြစ် အနိုင်ယူသွားသည်။\n\n\nကိုးကား'

In [8]:
wiki_ds['train'][402]['clean_text']

'EGS-zs8-1 သည် တောင်ဘက် နက္ခတ်တာရာ ဘိုးတီ ကြယ်စုတန်းမှ ဝေးကွာသည့် ဂလက်ဆီဖြစ်သည်။ ယင်းသည် ၂၀၁၅ မေလတွင် အဝေးဆုံးမှ ဂယ်လက်ဆီအဖြစ် ရပ်တည်လျက်ရှိခဲ့သည်။၂၀၁၅ ဂျူလိုင်၂၀ တွင် EGSY-2008532660 ဟူသော ဂလက်ဆီက အဝေးဆုံးအဖြစ် အနိုင်ယူသွားသည်။\nကိုးကား'

# 2. Facebook Flores

In [9]:
flores_ds = load_dataset("chuuhtetnaing/myanmar-facebook-flores-dataset")

In [10]:
flores_ds = flores_ds.map(cleanup_corpus, fn_kwargs={"column": "sentence"})

# 3. Myanmar Instruction Tuning

In [11]:
instruction_ds = load_dataset("chuuhtetnaing/myanmar-instruction-tuning-dataset")

In [12]:
instruction_content_ds = Dataset.from_dict({"content": instruction_ds["train"]["inputs"] + instruction_ds["train"]["targets"]})

In [13]:
instruction_content_ds = instruction_content_ds.map(cleanup_corpus, fn_kwargs={"column": "content"})

Map:   0%|          | 0/34714 [00:00<?, ? examples/s]

# 4. MM-LIB Book Corpus

In [14]:
book_corpus_ds = load_dataset("chuuhtetnaing/mm-lib-book-dataset")

In [15]:
book_corpus_ds

DatasetDict({
    train: Dataset({
        features: ['book_link', 'title', 'book_cover_image_link', 'category', 'author_name', 'author_description', 'raw_text'],
        num_rows: 437
    })
})

In [16]:
book_corpus_ds = book_corpus_ds.map(cleanup_corpus, fn_kwargs={"column": "raw_text"})

In [17]:
def check_zawgyi(row):
    lines = row['clean_text'].split("\n")[:300]
    
    zawgyi_count = 0
    for line in lines:
        zawgyi_count += is_zawgyi(line)
        
    if zawgyi_count > 0:
        return False
    
    return True

In [18]:
book_corpus_ds = book_corpus_ds.filter(check_zawgyi)

In [19]:
book_corpus_ds

DatasetDict({
    train: Dataset({
        features: ['book_link', 'title', 'book_cover_image_link', 'category', 'author_name', 'author_description', 'raw_text', 'clean_text'],
        num_rows: 421
    })
})

# Combine all clean text blocks

In [20]:
myanmar_text_blocks = []

myanmar_text_blocks.extend(wiki_ds["train"]["clean_text"])
myanmar_text_blocks.extend(flores_ds["train"]["clean_text"])
myanmar_text_blocks.extend(instruction_content_ds["clean_text"])
myanmar_text_blocks.extend(book_corpus_ds["train"]["clean_text"])

In [21]:
all_lines = []

for block in tqdm(myanmar_text_blocks):
    block_lines = block.split("\n")
    block_lines = [line.strip() for line in block_lines]
    block_lines = [line for line in block_lines if line != "" and is_zawgyi(line) < 0]
    
    all_lines.extend(block_lines)

100%|██████████| 153484/153484 [02:17<00:00, 1117.49it/s] 


In [22]:
print("Before breaking lines:", len(myanmar_text_blocks))
print("After breaking lines :", len(all_lines))

Before breaking lines: 153484
After breaking lines : 1746346


In [23]:
unique_lines = list(set(all_lines))

In [24]:
print("With duplication   :", len(all_lines))
print("Without duplication:", len(unique_lines))

With duplication   : 1746346
Without duplication: 1536661


In [25]:
unique_lines[:20]

['ဒန်းက အနည်းငယ်မကြိုက်သလိုလေသံဖြင့် သူက ဘာကိစ်္စ ဒီအထဲ ဝင်ချချင်ရတာလဲ၊ လူလည်ကြီးပဲဟု ဆိုသည်။ မက်အီဗွိုင်က အပြစ်တင်ဟန်လုပ်၍ ခေါင်းကိုယမ်းကာ မင်း အဲသလိုမပြောပါနဲ့ကွာ။ ဘယ်နှယ့် ညီနောင်အချင်းချင်း မေတ်္တာတရားဆိုတာ ဘယ်သွားထားမလဲ။ ဘယ်နှယ့် တို့လိုနေတာက ဘောင်းလ်လိုလူမျိုးကွ။သူက ဘုရားကျောင်းမှာ ပရိသတ်အပြည့်ကွ။ လျှံတောင်ထွက်နေသေးတာ။ ကင့် လမ်းမကြီးမှာလဲ ပရိသတ် ထောင်နဲ့ချီပြီးဟောနေတာဟု ဆို၍ ဖတ်လိုက်ပြန်သည်။',
 "အရေးတော်ပုံတွင်ပါဝင်ခဲ့ပြီးနောက် အခက်အခဲမျိုးစုံနှင့် ရင်ဆိုင်ခဲ့ရပြီး အဆိုးဝါးဆုံး အခက်အခဲမှာ ၎င်းဝါသနာပါသည့် စာရေးခြင်းကို နာမည်အရင်းဖြင့် ထုတ်ခွင့်မရခဲ့ခြင်းဖြစ်သည်။ 'ဂျင်မီ' အမည်ဖြင့် စာရေး ပါက အပိတ်ခံရမည်စိုးသောကြောင့် 'ဘုန်းသုခ' ကလောင်အမည်ဖြင့် ရတီမဂ္ဂဇင်းနှင့် အလင်းတန်းဂျာနယ်တို့တွင် ပင်တိုင် ရေးသားခဲ့သည်။",
 'သည်နေ့တော့ ဖိုးဝါ မနေ့ကထက် တွက်သားကိုက်တယ်။ မနေ့က ဘာရတာမှတ်လို့။ သုံးကျပ် တောင် နေစောင်းအောင် ရှာရတယ်။',
 'အောင် ကွပ်ကဲသော ကောင်းတုံခံမြို့အတွင်းမှ ခုခံသူတို့ကို ပြပါဆိုသောကြောင့် ဗလ မင်းထင်နှင့် စံလှကြီးတို့ကို ထုတ်ပြရ၏။',
 'ဩစတြေးလျ၊ နယူးဇီလန်နှင့် တောင်အာဖရိကရှိ တက္ကသိုလ်မ

In [26]:
len(unique_lines[1])

314

In [27]:
def create_break_pattern():
    """Creates and returns the regular expression pattern for Myanmar syllable breaking."""
    my_consonant = r"က-အ"
    # en_char = r"a-zA-Z0-9"

    other_char = r"ဣဤဥဦဧဩဪဿ၌၍၏၀-၉၊။"
    subscript_symbol = r'္'
    a_that = r'်'

    # Regular expression pattern for Myanmar syllable breaking
    return re.compile(
        r"((?<!" + subscript_symbol + r")[" + my_consonant + r"]"
        r"(?!["
        + a_that + subscript_symbol + r"])"
        + r"|[" + other_char + r"])"
    )

def break_syllables(line, break_pattern, separator):
    """Applie
    s syllable breaking rules to a line."""
    line = re.sub(r'\s+', ' ', line.strip()) # Normalize space
    segmented_line = break_pattern.sub(separator + r"\1", line)

    # Remove the leading delimiter if it exists
    if segmented_line.startswith(separator):
        segmented_line = segmented_line[len(separator):]

    # Replace delimiter+space+delimiter with a single space
    double_delimiter = separator + " " + separator
    segmented_line = segmented_line.replace(double_delimiter, " ")

    return segmented_line

break_pattern = create_break_pattern()

def mm_split(text):
    text = text.strip()
    seperator = "|X|"

    result = break_syllables(text, break_pattern, seperator)
    result = result.split(seperator)

    return result

In [28]:
len(mm_split(unique_lines[1]))

106

In [29]:
import random

In [30]:
random.shuffle(unique_lines)

In [31]:
one_liner = " ".join(unique_lines)

In [32]:
words = mm_split(one_liner)

In [33]:
new_words = []
for word in words:
    if len(word) > 30:
        splitted_words = word.split(" ")
        new_words.extend([w for w in splitted_words if w.strip() != ""])
    else:
        new_words.append(word)

In [34]:
len(words), len(new_words)

(79728640, 80431224)

In [35]:
one_liner[:100]

'ခေတ်အခါကလည်းမကောင်းတော့သူ့ကိုပထမရိုးရိုးမေးကြတယ်...နောက်တော့မညှာကြတော့ဘူး...အုပ်ချုပ်ရေးတာဝန်ခံဆီပို'

In [36]:
"".join(new_words[:50])

'ခေတ်အခါကလည်းမကောင်းတော့သူ့ကိုပထမရိုးရိုးမေးကြတယ်...နောက်တော့မညှာကြတော့ဘူး...အုပ်ချုပ်ရေးတာဝန်ခံဆီပို့လိုက်ကြတယ်။ MMR013 Yangon ဦးဉာဏ်ထွန်း ( ၃ ဩဂုတ် ၂၀၂၃ - လက်ရှိ)Kyauktaw'

In [37]:
def split_by_sentence_lengths(text):
    total_chars = len(text)
    sentence_lengths = list(range(50, 0, -1))  # From 50 to 1
    sentence_percent = 0.02  # 2%

    result = []

    idx = 0  # Start index

    for length in tqdm(sentence_lengths):
        chars_for_this_length = int(total_chars * sentence_percent)
        num_sentences = chars_for_this_length // length

        for _ in range(num_sentences):
            if idx + length <= total_chars:
                result.append("".join(text[idx:idx+length]))
                idx += length
            else:
                break

    return result

In [38]:
corpus = split_by_sentence_lengths(new_words)

100%|██████████| 50/50 [00:02<00:00, 23.23it/s]


In [39]:
len(corpus)

7237512

In [40]:
with open("./my_corpus_4_datasets.txt", "w") as file:
    file.write("\n".join(corpus))

In [ ]:
import itertools

In [ ]:
single_words = []

for line in tqdm(unique_lines):
    words = mm_split(line)
    words = [word.split(" ") for word in words]
    words = list(itertools.chain(*words))
    words = [word.strip() for word in words]
    words = [word for word in words if word != ""]
    single_words.extend(words)

100%|██████████| 860961/860961 [00:26<00:00, 31888.46it/s]


In [ ]:
for line in unique_lines:
    if "ကြၽႏ္ုပ္ကိုယ္တိုင္ေ" in line:
        print(line)

In [ ]:
unique_single_words = list(set(single_words))

In [ ]:
print("All single words       :", len(single_words))
print("All single unique words:", len(unique_single_words))

All single words       : 48136074
All single unique words: 72968


In [ ]:
unique_single_words[-200:-150]

['Snare',
 'OLD',
 'စိန်ြ',
 '26;',
 'Pad',
 'နိဗ္ဗူ',
 'bags,',
 'succulent',
 'Can',
 'ဿဂ္ဂီ',
 '(Latin)',
 '170',
 'စ)(Great',
 'ready,',
 'walking,',
 'develop.',
 '~~~~~~~~~~~~~~~~',
 'သီ..',
 'else,',
 'ocean,',
 'Apollo)',
 'Rosalbu',
 '၏,',
 'Museum.',
 '............."',
 'ဝတ်ေ',
 'sub',
 'ဆိုင်း-',
 'studying',
 'ကဏ်',
 'Zeyarthu',
 'Practice)',
 'ထက်(',
 'Square)',
 '(Suttanipta,',
 'မှာ--',
 'honour',
 'မှာ-',
 '(electric',
 'wert,',
 'unarmed',
 '2Mahdukkhakkhandha-sutta,',
 '[22]',
 'ခင်ဗ်ာ"',
 '306om',
 'Sony',
 'ဟိယ်',
 'Approval',
 'ဖောင်း"',
 '(Darwins']

In [ ]:
unique_words = []
for word in tqdm(single_words):
    if word not in unique_words:
        unique_words.append(word)

100%|██████████| 48136074/48136074 [02:42<00:00, 295795.69it/s]


In [ ]:
len(unique_words)

72968

In [ ]:
unique_words[-100:]

['ဇာန်',
 'ဟော......',
 'ဒေါ့(',
 'ဖို်င်',
 'General.',
 'စို့သ္မီး',
 '(IDPsRefugee',
 'Relief',
 'Committee-IRRC)',
 'Wunpawng',
 'Ninghtoi',
 'BRIDGE',
 'KWAT',
 'တာ်း',
 'ပေါ်ှ',
 'သု့န်',
 '(Fair',
 'Rent)',
 'တ.်',
 '223',
 'images/084814E.png',
 'ရိမ္မိ',
 'Plastic',
 'လေမ်း',
 'ကျေက်ာ',
 'yatana,',
 'Sphere.',
 'spheres:',
 'conceptions).',
 '၈က်',
 'ကြတ္ထ',
 '1930',
 'ဟက်*',
 'တဲ့မ္ဘာ',
 '......................................................................................................................................................................',
 '(Inca',
 '(Trujillo)',
 '(Haity)',
 '(Dominican',
 '(Caribbean',
 'နဲ့ဇ်',
 'Nunez',
 'Balboa)',
 '(Panama)',
 '(Pascual',
 'Andagoya)',
 '(Llama)',
 '(Indian)',
 'V)',
 '(Cajamarca)',
 '(Atahualpa)',
 "ဖြုတ်''",
 'ကားရ်',
 '(Cause)',
 "Kafka's",
 'novels',
 'cause.)',
 'Nile,',
 'world?',
 '450)',
 'BC',
 'စာခ်',
 '(Thank',
 'တစ်ရ်',
 'ဝင်္ကာ',
 'Discovery',
 'မျို=',
 'ဘွား,',
 'scholarly',
 'renowned',
 'themselvesmoney,

In [ ]:
unique_words

['မ',
 'ခင်',
 'လေး',
 'ယဉ်',
 'သည်',
 'ဝံ့',
 'တ',
 'နှ',
 'င့်',
 'ကို',
 'အောင်',
 'လွင်',
 '၏',
 'မျက်',
 'နှာ',
 'လှမ်း',
 'ကြ',
 'ည့်',
 'ရင်း',
 'သိမ်',
 'မွေ့',
 'စွာ',
 'ပြန်',
 'ပြော',
 'လေ',
 '။',
 'ထို',
 'အ',
 'ရင်',
 'ထဲ',
 'ကြည်',
 'နူး',
 'သာ',
 'ယာ',
 'စီး',
 'ဝင်',
 'လာ',
 'တိုင်း',
 'ဆို',
 'လျှင်',
 'ဖွား',
 'တို့',
 'တော',
 'သူ',
 'တွေး',
 'ထင်',
 'ခဲ့',
 'သ',
 'လို',
 'ခြောက်',
 'သွေ့',
 'ငြီး',
 'ငွေ့',
 'ဖွယ်',
 'ရှိ',
 'တော့',
 'မည်',
 'ဟုတ်',
 'ပေ',
 'ပန်း',
 'ကြွေ',
 'စမ်း',
 'ရေ',
 'သံ',
 'လျက်',
 'ဒီ',
 'မယ်',
 'ငါ့',
 'ညီ',
 'ယ့်',
 'ကိုယ်',
 'သတ်',
 'သေ',
 'တာ',
 'ဟာ',
 'ဘ',
 'ဝ',
 'ရှုံး',
 'ပေး',
 'ရာ',
 'ရောက်',
 'တယ်',
 'ယောက်ျား',
 'ဆန်',
 'ဘူး',
 'ရုန်း',
 'ကန်',
 'ဆိုင်',
 'သက်',
 'စွ',
 'န့်',
 'သွား',
 'က',
 'စ',
 'မြတ်',
 'သေး',
 'မျှ',
 'မှန်',
 'ကြိုး',
 'စား',
 'နိုင်',
 'ပါ',
 'နောက်',
 'ပြီး',
 'ကမ္ဘာ',
 'ကြီး',
 'ကျယ်',
 'ကြာ',
 'မှု',
 'ပေါ်',
 'မင်း',
 'လည်း',
 'ကွင်း',
 'လုံး',
 'ကျွတ်',
 'လွတ်',
 'မှာ',
 'ပြေ',
 'သိမ်း',
 '၊',
 'အိမ်',

In [ ]:
len(single_words)

50953791

In [ ]:
unique_single_words

['သောThe',
 'ရီခ်',
 'လက္မ်ားႏွင့္',
 'မ႔',
 'မာန္ဆိုးျ',
 'လာလ်က္က',
 'ကီးႏွစ္သြယ္',
 'ပ်က္လာ',
 'သာက္ခိုင္းေ',
 'သစ္ငုတ္တိုမ်ားေ',
 '(Athenian',
 'ပန္မက္လာျ',
 'ကားက်',
 'လွင်)-',
 'ပိန္က',
 'နာက္တတ္',
 'ချယ်"',
 'ဆွြဲ',
 'လြတ္ပါေ',
 'ဆိုဒ်(Outside)',
 'ပြားခ်င္တယ္',
 'ပြတ္လ်က္',
 'အင်)',
 'ကာက္တာ',
 'ဆြတ္လုိက္ေ',
 'ထုတ္ခဲ့ၿ',
 'တ႐ွင္',
 'ရွယ္လစ္အေျ',
 'လက္တစ္ဖက္ကမ္း',
 'စည္းႏွင့္',
 'ကန္ထုတ္တဲ့ေ',
 'ထာက္',
 'စောင်း.',
 'ဟွန်း)',
 'proposal,',
 'သိခ်င္ပုံ',
 'Partly',
 'Royalty',
 'တိုင္ေ',
 'အုပ္မ်ား',
 'ကြပ္ကဲလ်က္',
 'statues',
 'ဆယ္လ',
 'ရွင္သတ္မွတ္ေ',
 'ဘန်း',
 'ဟတ်(ဖ်)',
 'Equi)',
 'တိုုင်း',
 'ကက္႐ိုးႏွစ္တုံး',
 'strike',
 'ရီ႐ွိ',
 'immeasurably',
 'ရးလ္ဆန္႔',
 'parents.',
 '၀)...',
 'Mae,',
 'တခ်ိဳက္တည္း',
 'ဖာ္အျ',
 'ဇင္ကေ',
 'index-8_1.jpg',
 'ကြၽတ္က်ေ',
 'ပန္ဝင္က',
 'talk',
 'Roof',
 '(UAE)',
 'ဆဲၾ',
 '(electrical',
 'အက်ိဳး႐ွိႏိုင္သည္',
 'ဂ(',
 'မည္ဟီး',
 'ကဏ္ဌာ',
 'အင္ႏွစ္မ်ိဳး',
 'ည့ွံ',
 "ဏှာ'",
 'သက္သာခ်င္ေ',
 'movie',
 'မွာင္ပိန္းလ်က္',
 'သည္တိုက္ပြဲ',
 'ကုင်္ကု',
 '

In [ ]:
unique_single_words

['သောThe',
 'ရီခ်',
 'လက္မ်ားႏွင့္',
 'မ႔',
 'မာန္ဆိုးျ',
 'လာလ်က္က',
 'ကီးႏွစ္သြယ္',
 'ပ်က္လာ',
 'သာက္ခိုင္းေ',
 'သစ္ငုတ္တိုမ်ားေ',
 '(Athenian',
 'ပန္မက္လာျ',
 'ကားက်',
 'လွင်)-',
 'ပိန္က',
 'နာက္တတ္',
 'ချယ်"',
 'ဆွြဲ',
 'လြတ္ပါေ',
 'ဆိုဒ်(Outside)',
 'ပြားခ်င္တယ္',
 'ပြတ္လ်က္',
 'အင်)',
 'ကာက္တာ',
 'ဆြတ္လုိက္ေ',
 'ထုတ္ခဲ့ၿ',
 'တ႐ွင္',
 'ရွယ္လစ္အေျ',
 'လက္တစ္ဖက္ကမ္း',
 'စည္းႏွင့္',
 'ကန္ထုတ္တဲ့ေ',
 'ထာက္',
 'စောင်း.',
 'ဟွန်း)',
 'proposal,',
 'သိခ်င္ပုံ',
 'Partly',
 'Royalty',
 'တိုင္ေ',
 'အုပ္မ်ား',
 'ကြပ္ကဲလ်က္',
 'statues',
 'ဆယ္လ',
 'ရွင္သတ္မွတ္ေ',
 'ဘန်း',
 'ဟတ်(ဖ်)',
 'Equi)',
 'တိုုင်း',
 'ကက္႐ိုးႏွစ္တုံး',
 'strike',
 'ရီ႐ွိ',
 'immeasurably',
 'ရးလ္ဆန္႔',
 'parents.',
 '၀)...',
 'Mae,',
 'တခ်ိဳက္တည္း',
 'ဖာ္အျ',
 'ဇင္ကေ',
 'index-8_1.jpg',
 'ကြၽတ္က်ေ',
 'ပန္ဝင္က',
 'talk',
 'Roof',
 '(UAE)',
 'ဆဲၾ',
 '(electrical',
 'အက်ိဳး႐ွိႏိုင္သည္',
 'ဂ(',
 'မည္ဟီး',
 'ကဏ္ဌာ',
 'အင္ႏွစ္မ်ိဳး',
 'ည့ွံ',
 "ဏှာ'",
 'သက္သာခ်င္ေ',
 'movie',
 'မွာင္ပိန္းလ်က္',
 'သည္တိုက္ပြဲ',
 'ကုင်္ကု',
 '